# FORNAX 3D Neutrino Spectra

Some I/O and plotting using the Fornax 3D models from [Vartanyan, Burrows, et al., MNRAS 482(1):351, 2019](https://arxiv.org/abs/1809.05106), which express the angular dependence of the neutrino luminosity from CCSNe in terms of a spherical harmonic expansion up to order $\ell=2$.

The data are available on [the Burrows group website](https://www.astro.princeton.edu/~burrows/nu-emissions.3d/).

In [1]:
from snewpy.neutrino import Flavor
from snewpy.models import Fornax_2019_3D

from astropy import units as u

from glob import glob

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
mpl.rc('font', size=16)

## Test Instantiation using a $10M_{\odot}$ Model

In [3]:
model = Fornax_2019_3D('../../models/Fornax_2019/lum_spec_10M.h5')
model

**Fornax 3D Model**: ../../models/Fornax_2019/lum_spec_10M.h5

|Parameter|Value|
|:---------|:-----:|
|Progenitor mass | $10$ $\mathrm{M_{\odot}}$|

In [5]:
energy, dE, spectra = model._get_binnedspectra(0.5*u.s, 55*u.deg, 32*u.deg)
energy[Flavor.NU_E], spectra[Flavor.NU_E]

(<Quantity [  1.24497987,   2.00257642,   3.2211865 ,   5.18134656,
              8.33430544,  13.40590643,  21.56368381,  34.68564113,
             55.79258678,  89.74355493, 144.3544047 , 232.19711067] MeV>,
 <Quantity [4.82964294e+48, 1.84378966e+49, 1.32565946e+50, 4.59221029e+50,
            9.15665038e+50, 1.09921094e+51, 6.07187503e+50, 1.30073448e+50,
            5.85847804e+48, 2.78255236e+46, 7.84490124e+44, 1.12724333e+43] erg / (MeV s)>)

In [ ]:
t = 600 * u.ms
E = np.arange(0,101) * u.MeV
theta = 20*u.degree
phi = 20*u.degree

spectra = model.get_initialspectra(t, E, theta, phi)

In [ ]:
lines = ['--', '-.', '-', ':']
for line, flavor in zip(lines, Flavor):
    plt.plot(E, spectra[flavor], ls=line, label=flavor.to_tex())
plt.legend()

## Test Caching of Full Model Angular Dependence

In [ ]:
model = Fornax_2019_3D('../../models/Fornax_2019/lum_spec_10M.h5', cache_flux=True)
model

In [ ]:
import healpy as hp

fig, axes = plt.subplots(5,3, figsize=(8*3, 5*4.5))

nt = len(model.time)
L_nue = model.luminosity[Flavor.NU_E].to('1e52*erg/s')
L_nue_bar = model.luminosity[Flavor.NU_E_BAR].to('1e52*erg/s')
L_nux = model.luminosity[Flavor.NU_X].to('1e52*erg/s')

for i in range(5):
    j = i*nt//5
    
    ax = axes[i,0]
    plt.axes(ax)
    hp.mollview(L_nue[j], title=r'$L_{{\nu_e}}(t={:.3f})$'.format(model.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='viridis',
                hold=True)
    
    ax = axes[i,1]
    plt.axes(ax)
    hp.mollview(L_nue_bar[j], title=r'$L_{{\bar{{\nu}}_e}}(t={:.3f})$'.format(model.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='magma',
                hold=True)
    
    ax = axes[i,2]
    plt.axes(ax)
    hp.mollview(L_nux[j], title=r'$L_{{\nu_X}}(t={:.3f})$'.format(model.time[j]),
                unit=r'$10^{52}$ erg s$^{-1}$', cmap='plasma',
                hold=True)

### Plot $L_\nu(t,\theta,\varphi)$ at a Particular Location

In [ ]:
k = 55

fig, ax = plt.subplots(1,1, figsize=(10,5), tight_layout=True)
ax.plot(model.time, L_nue[:,k], lw=2, label=r'$\nu_e$')
ax.plot(model.time, L_nue_bar[:,k], lw=2, label=r'$\bar{\nu}_e$')
ax.plot(model.time, L_nux[:,k], lw=2, label=r'$\nu_X$')
ax.set(xlabel='time [s]',
       xlim=model.time[0::len(model.time)-1].value,
       ylabel=r'$L_{\nu}(t,\theta,\varphi)$ [$10^{52}$ erg s$^{-1}$]',
       ylim=(0, 1.1*np.max(L_nue).value))
ax.grid(ls=':')
ax.legend(fontsize=12);

### Superimpose $L_\nu(t,\theta,\varphi)$ at All Locations on the Sphere

Plot $L(t)$ at all locations as well as the average, and then the deviation from the average.

In [ ]:
Lavg = np.average(L_nue, axis=1)
dL_over_L = (L_nue - Lavg[:,np.newaxis]) / Lavg[:,np.newaxis]

fig, axes = plt.subplots(2,1, figsize=(10,8),
                         gridspec_kw = {'height_ratios':[3,1], 'hspace':0},
                         sharex=True, tight_layout=True)

ax = axes[0]
ax.plot(model.time, L_nue, color='gray', alpha=0.05)
ax.plot(model.time, Lavg, color='r', ls='--')
ax.set(xlim=model.time[0::len(model.time)-1].value,
       ylabel=r'$L_{\nu_e}(t,\theta,\varphi)$ [$10^{52}$ erg s$^{-1}$]',
       ylim=(0, 1.1*np.max(L_nue).value),
       title=r'$L_{\nu_e}(t,\theta,\varphi)$')
ax.grid(ls=':')

ax = axes[1]
ax.plot(model.time, dL_over_L, color='gray', alpha=0.1)
ax.plot(model.time, np.zeros(model.time.shape), ls='--', color='r')
ax.set(xlabel='time [s]',
       ylabel=r'$(L_\nu - \bar{L}_\nu)/\bar{L}_\nu$',
       ylim=(-0.3, 0.3))
ax.grid(ls=':');

In [ ]:
energy, spectra = model._get_binnedspectra(0.5*u.s, 55*u.deg, 32*u.deg)
np.log10(energy[Flavor.NU_E])

In [ ]:
j, k = 200, 73

In [ ]:
model.dLdE[flavor][j,:,k].shape

In [ ]:
model.E